In [ ]:
!pip install "numpy>=1.23.5,<2.0.0" "pandas>=2.2.3" "matplotlib>=3.10.1" "tensorflow>=2.15.0" "scikit-learn>=1.3.0" "pickle"

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def generate_synthetic_mobility_data(num_users=100, days=7, sampling_rate_seconds=10):
    # Define cell tower locations (grid-based for simplicity)
    grid_size = 10
    cell_towers = []
    for i in range(grid_size):
        for j in range(grid_size):
            cell_towers.append({
                'id': f'cell_{i}_{j}',
                'x': i * 1000,  # 1km spacing
                'y': j * 1000,
                'coverage_radius': 600  # meters
            })
    
    # Create mobility patterns
    data = []
    start_time = datetime.now()
    
    for user_id in range(num_users):
        # Choose a movement pattern type for this user
        pattern_type = np.random.choice(['commuter', 'random_walk', 'stationary', 'high_mobility'])
        
        # Set starting position
        if pattern_type == 'commuter':
            # Commuters typically start at home locations
            x = np.random.uniform(0, 3000)
            y = np.random.uniform(0, 3000)
        else:
            # Random starting position within the grid
            x = np.random.uniform(0, grid_size * 1000)
            y = np.random.uniform(0, grid_size * 1000)
        
        # Generate movement for each timestep
        current_time = start_time
        velocity = np.random.uniform(0.5, 2.0)  # base velocity in m/s
        
        if pattern_type == 'stationary':
            velocity = 0.1  # mostly stationary with small movements
        elif pattern_type == 'high_mobility':
            velocity = np.random.uniform(10, 30)  # high speed (e.g., vehicle)
        
        heading = np.random.uniform(0, 2*np.pi)  # initial direction
        
        for day in range(days):
            for hour in range(24):
                # Adjust behavior based on time of day
                if pattern_type == 'commuter':
                    # Morning commute (7-9 AM)
                    if 7 <= hour < 9:
                        # Head to work area
                        target_x = np.random.uniform(7000, 9000)
                        target_y = np.random.uniform(7000, 9000)
                        heading = np.arctan2(target_y - y, target_x - x)
                        velocity = np.random.uniform(5, 15)  # faster during commute
                    # Evening commute (5-7 PM)
                    elif 17 <= hour < 19:
                        # Head back home
                        target_x = np.random.uniform(0, 3000)
                        target_y = np.random.uniform(0, 3000)
                        heading = np.arctan2(target_y - y, target_x - x)
                        velocity = np.random.uniform(5, 15)
                    # At work or at home
                    else:
                        velocity = np.random.uniform(0, 0.5)  # minimal movement
                        # Random direction changes
                        if np.random.random() < 0.1:
                            heading = np.random.uniform(0, 2*np.pi)
                
                # Generate positions for this hour
                samples_per_hour = 3600 // sampling_rate_seconds
                for s in range(samples_per_hour):
                    # Add some randomness to movement
                    if np.random.random() < 0.2:
                        heading += np.random.uniform(-0.5, 0.5)
                    
                    # Update position
                    dx = velocity * sampling_rate_seconds * np.cos(heading)
                    dy = velocity * sampling_rate_seconds * np.sin(heading)
                    
                    x += dx
                    y += dy
                    
                    # Ensure within boundaries
                    x = max(0, min(x, grid_size * 1000))
                    y = max(0, min(y, grid_size * 1000))
                    
                    # Find closest cell tower
                    distances = [np.sqrt((x - tower['x'])**2 + (y - tower['y'])**2) for tower in cell_towers]
                    connected_cell_idx = np.argmin(distances)
                    connected_cell = cell_towers[connected_cell_idx]
                    signal_strength = calculate_signal_strength(distances[connected_cell_idx])
                    
                    # Find potential handover candidates
                    handover_candidates = []
                    for idx, dist in enumerate(distances):
                        if idx != connected_cell_idx and dist < connected_cell['coverage_radius'] * 1.5:
                            candidate_strength = calculate_signal_strength(dist)
                            handover_candidates.append({
                                'cell_id': cell_towers[idx]['id'],
                                'signal_strength': candidate_strength
                            })
                    
                    # Determine if handover is needed
                    handover_needed = False
                    handover_target = None
                    for candidate in handover_candidates:
                        if candidate['signal_strength'] > signal_strength + 3:  # 3dB hysteresis
                            handover_needed = True
                            handover_target = candidate['cell_id']
                            break
                    
                    # Record data point
                    timestamp = current_time + timedelta(days=day, hours=hour, seconds=s*sampling_rate_seconds)
                    data.append({
                        'timestamp': timestamp,
                        'user_id': f'user_{user_id}',
                        'x': x,
                        'y': y,
                        'velocity': velocity,
                        'heading': heading,
                        'connected_cell': connected_cell['id'],
                        'signal_strength': signal_strength,
                        'handover_needed': handover_needed,
                        'handover_target': handover_target,
                        'pattern_type': pattern_type
                    })
    
    return pd.DataFrame(data)

def calculate_signal_strength(distance):
    # Simple path loss model: -70dBm at 100m, -10dB per distance doubling
    base_signal = -70
    if distance == 0:
        return base_signal
    
    # Log-distance path loss model
    path_loss = 20 * np.log10(distance / 100)
    return base_signal - path_loss

# Generate dataset
mobility_data = generate_synthetic_mobility_data(num_users=100, days=3)

In [ ]:
def add_network_conditions(df):
    # Add time-of-day based network congestion
    df['hour'] = df['timestamp'].dt.hour
    
    # Peak hours have more congestion
    peak_hours_morning = (7 <= df['hour']) & (df['hour'] <= 9)
    peak_hours_evening = (16 <= df['hour']) & (df['hour'] <= 19)
    
    # Base network load (30-50%)
    df['network_load'] = np.random.uniform(0.3, 0.5, size=len(df))
    
    # Increase during peak hours (60-90%)
    df.loc[peak_hours_morning | peak_hours_evening, 'network_load'] = \
        np.random.uniform(0.6, 0.9, size=len(df[peak_hours_morning | peak_hours_evening]))
    
    # Add random network quality metrics
    df['sinr'] = df['signal_strength'] - np.random.uniform(-5, 5, size=len(df)) - 10 * df['network_load']
    df['throughput_mbps'] = 10 * (1 + np.log10(1 + df['sinr'])) * (1 - df['network_load']*0.7)
    
    # Add UE capabilities
    device_categories = ['5G_basic', '5G_advanced', '5G_premium']
    df['device_type'] = np.random.choice(device_categories, size=len(df))
    
    # Add handover performance metrics
    df['handover_latency'] = np.random.uniform(10, 30, size=len(df))  # ms
    
    # Some percentage of handovers fail
    df['handover_success'] = np.random.choice([True, False], size=len(df), p=[0.95, 0.05])
    
    return df

mobility_data = add_network_conditions(mobility_data)

# Save the generated data
try:
    # Save as parquet
    mobility_data.to_parquet('mobility_data.parquet')
    print("Successfully saved data to mobility_data.parquet")
    
    # Save as pickle
    with open('mobility_data.pkl', 'wb') as f:
        pickle.dump(mobility_data, f)
    print("Successfully saved data to mobility_data.pkl")
except Exception as e:
    print(f"Error saving data: {str(e)}")